In [38]:
# Utilities
import fiona
from shapely.geometry import shape, mapping  # Ensure this is imported
import os
import datetime
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import requests
import getpass

from sentinelhub import (
    SHConfig,
    DataCollection,
    SentinelHubCatalog,
    SentinelHubRequest,
    BBox,
    bbox_to_dimensions,
    CRS,
    MimeType,
    Geometry,
)

from utils import plot_image

Load json or txt file

In [39]:
with open('credentials_S1_imagery.txt', 'r') as f:
    credentials = dict(line.strip().split('=', 1) for line in f if '=' in line)
# Access the credentials
password = credentials.get('COPERNICUS_PASSWORD')
username = credentials.get('COPERNICUS_USERNAME')
aoi_path = credentials.get(r'AOIPATH')
start_date = credentials.get('STARTDATE')
end_date = credentials.get('ENDDATE') 
output_name = credentials.get('OUTPUTNAME')
data_collection = "SENTINEL-1"

In [40]:
output_name

'S1_IMAGEOUTPUT'

In [41]:
import fiona
from shapely.geometry import shape, mapping, Polygon

# Open the shapefile or GeoPackage
# Open the shapefile or GeoPackage
with fiona.open(aoi_path) as shp:
    for feature in shp:
        geometry = shape(feature["geometry"])

        # Check if the geometry has Z and strip it
        if geometry.has_z:
            # Create a 2D version of the geometry
            geometry_2d = Polygon([(x, y) for x, y, z in geometry.exterior.coords])
            wkt = geometry_2d.wkt

            # Adjust WKT format to match the desired style
            wkt_fixed = wkt.replace('POLYGON (', 'POLYGON(').replace(', ', ',').replace(' ', ' ')
          

aoi = wkt_fixed
aoi = f"{aoi}'"
print(aoi)

POLYGON((-1.06482362615344 51.4775993109376,-1.0648176309286 51.4767329603795,-1.06360890264833 51.4766474543255,-1.0635910538134 51.4775435502895,-1.06482362615344 51.4775993109376))'


In [42]:
json = requests.get(
    f"https://catalogue.dataspace.copernicus.eu/odata/v1/Products?&$filter=((Collection/Name%20eq%20%27SENTINEL-1%27%20and%20(Attributes/OData.CSC.StringAttribute/any(att:att/Name%20eq%20%27instrumentShortName%27%20and%20att/OData.CSC.StringAttribute/Value%20eq%20%27SAR%27)%20and%20(contains(Name,%27GRD%27)%20and%20not%20contains(Name,%27_COG%27)%20and%20OData.CSC.Intersects(area=geography%27SRID=4326;{aoi})))%20and%20Online%20eq%20true)%20and%20ContentDate/Start%20ge%20{start_date}T00:00:00.000Z%20and%20ContentDate/Start%20lt%20{end_date}T23:59:59.999Z)&$orderby=ContentDate/Start%20desc&$expand=Attributes&$count=True&$top=50&$expand=Assets&$skip=0"
).json()
df = pd.DataFrame.from_dict(json["value"]).head(5)
df_all = pd.DataFrame.from_dict(json["value"])
df

,@odata.mediaContentType,Id,Name,ContentType,ContentLength,OriginDate,PublicationDate,ModificationDate,Online,EvictionDate,S3Path,Checksum,ContentDate,Footprint,GeoFootprint,Attributes,Assets
0,application/octet-stream,000da373-5564-401c-9159-8775c947866c,S1A_IW_GRDH_1SDV_20250130T175749_20250130T1758...,application/octet-stream,1754987003,2025-01-30T18:39:34.252000Z,2025-01-30T18:43:10.034495Z,2025-01-30T18:43:10.034495Z,True,9999-12-31T23:59:59.999999Z,/eodata/Sentinel-1/SAR/IW_GRDH_1S/2025/01/30/S...,"[{'Value': '2daf0f7ca02dfa78bdafdc98b94990ca',...","{'Start': '2025-01-30T17:57:49.436317Z', 'End'...",geography'SRID=4326;POLYGON ((-3.710244 49.690...,"{'type': 'Polygon', 'coordinates': [[[-3.71024...","[{'@odata.type': '#OData.CSC.StringAttribute',...","[{'Type': 'QUICKLOOK', 'Id': 'e3d87f6c-7ac5-47..."
1,application/octet-stream,653c2891-45cf-4ea4-b4e0-5330c217f403,S1A_IW_GRDH_1SDV_20250127T062322_20250127T0623...,application/octet-stream,1780263598,2025-01-27T07:21:09.719000Z,2025-01-27T07:23:33.172684Z,2025-01-27T07:23:57.009033Z,True,9999-12-31T23:59:59.999999Z,/eodata/Sentinel-1/SAR/IW_GRDH_1S/2025/01/27/S...,"[{'Value': 'f10d564cfa6b6fb3d7be8c1b3ec28660',...","{'Start': '2025-01-27T06:23:22.271723Z', 'End'...",geography'SRID=4326;POLYGON ((-0.696136 50.983...,"{'type': 'Polygon', 'coordinates': [[[-0.69613...","[{'@odata.type': '#OData.CSC.StringAttribute',...","[{'Type': 'QUICKLOOK', 'Id': 'ef6100fd-b47f-4d..."
2,application/octet-stream,5a422d7b-1b26-4646-93d4-b7893d659f13,S1A_IW_GRDH_1SDV_20250125T174939_20250125T1750...,application/octet-stream,1748008477,2025-01-25T18:31:11.348000Z,2025-01-25T18:34:02.646160Z,2025-01-25T18:34:30.409952Z,True,9999-12-31T23:59:59.999999Z,/eodata/Sentinel-1/SAR/IW_GRDH_1S/2025/01/25/S...,"[{'Value': 'fbad15a6ac5c04d9b63da5921529f02e',...","{'Start': '2025-01-25T17:49:39.457472Z', 'End'...",geography'SRID=4326;POLYGON ((-1.699804 49.884...,"{'type': 'Polygon', 'coordinates': [[[-1.69980...","[{'@odata.type': '#OData.CSC.StringAttribute',...","[{'Type': 'QUICKLOOK', 'Id': '969b09f0-a2bb-45..."
3,application/octet-stream,a83cf32a-2b3c-4ffc-ab29-bfe3a6d1708d,S1A_IW_GRDH_1SDV_20250122T061527_20250122T0615...,application/octet-stream,1778833262,2025-01-22T07:15:44.849000Z,2025-01-22T07:18:21.132077Z,2025-01-22T07:18:47.439049Z,True,9999-12-31T23:59:59.999999Z,/eodata/Sentinel-1/SAR/IW_GRDH_1S/2025/01/22/S...,"[{'Value': '6456ded1c2dedcf531e18b5bbcf9671e',...","{'Start': '2025-01-22T06:15:27.513923Z', 'End'...",geography'SRID=4326;POLYGON ((1.001044 49.8530...,"{'type': 'Polygon', 'coordinates': [[[1.001044...","[{'@odata.type': '#OData.CSC.StringAttribute',...","[{'Type': 'QUICKLOOK', 'Id': '6beb4e99-ba9e-49..."
4,application/octet-stream,d45793bf-9e47-46aa-bfc6-9b26f79ba381,S1A_IW_GRDH_1SDV_20250115T062322_20250115T0623...,application/octet-stream,1780329932,2025-01-15T07:27:55.828000Z,2025-01-15T07:31:11.108730Z,2025-01-15T07:31:39.432379Z,True,9999-12-31T23:59:59.999999Z,/eodata/Sentinel-1/SAR/IW_GRDH_1S/2025/01/15/S...,"[{'Value': 'ec21a6f42a20e507ab879d3c7f55bf3b',...","{'Start': '2025-01-15T06:23:22.726927Z', 'End'...",geography'SRID=4326;POLYGON ((-0.695427 50.983...,"{'type': 'Polygon', 'coordinates': [[[-0.69542...","[{'@odata.type': '#OData.CSC.StringAttribute',...","[{'Type': 'QUICKLOOK', 'Id': '5e8238af-74fb-48..."


In [46]:
df2 = df.iloc[1]['Id']
df2

'653c2891-45cf-4ea4-b4e0-5330c217f403'

In [47]:
def get_access_token(username: str, password: str) -> str:
    data = {
        "client_id": "cdse-public",
        "username": username,
        "password": password,
        "grant_type": "password",
    }
    try:
        r = requests.post(
            "https://identity.dataspace.copernicus.eu/auth/realms/CDSE/protocol/openid-connect/token",
            data=data,
        )
        r.raise_for_status()
    except Exception as e:
        raise Exception(
            f"Access token creation failed. Response from the server was: {r.json()}"
        )
    return r.json()["access_token"]

# Automatically use credentials from the file
access_token = get_access_token(username, password)

In [48]:
url = f"https://zipper.dataspace.copernicus.eu/odata/v1/Products({df2})/$value"

headers = {"Authorization": f"Bearer {access_token}"}

session = requests.Session()
session.headers.update(headers)
response = session.get(url, headers=headers, stream=True)

print(f'{output_name} is currently being downloaded')

with open(f"{output_name}.zip", "wb") as file:
    for chunk in response.iter_content(chunk_size=8192):
        if chunk:
            file.write(chunk)

print(f'{output_name} has now been exported as a .zip file in your working directory') 

S1_IMAGEOUTPUT is currently being downloaded
S1_IMAGEOUTPUT has now been exported as a .zip file in your working directory
